In [1]:
import sys
sys.path.append('../')
import exp
import regression as r

In [2]:
df = exp.get_wp_energy_data()
df.head()

,PET,VPD,air_temp,doy,precip,soil_temp,sw_in,wind_speed,year,wp_RNET,...,wp_bnd3,wp_bnd7,wp_evi,wp_lswi2,wp_ndvi,wp_LST.day,wp_LST.night,LW_IN.wp,LW_IN.si,LW_IN
0,7.33,0.808731,19.179167,195,0.0,22.320833,30.3156,4.958333,2012,20.798342,...,0.029934,0.112914,0.335343,0.354057,6052.592985,31.567899,17.204530,29.105844,24.562860,29.105844
1,6.52,0.755945,19.325000,196,0.0,21.770833,29.6316,3.791667,2012,20.573593,...,0.029932,0.112904,0.335286,0.354033,6178.213349,29.570000,17.390000,29.760736,26.096551,29.760736
2,6.92,0.858993,20.262500,197,0.0,21.908333,29.3472,4.137500,2012,20.475931,...,0.029930,0.112893,0.335229,0.354008,6313.892022,31.097908,17.235624,30.299442,25.986418,30.299442
3,6.35,0.477617,16.791667,198,0.0,22.420833,28.8180,6.033333,2012,20.571045,...,0.029928,0.112882,0.335173,0.353984,6459.629005,30.868718,17.248525,28.978868,25.122209,28.978868
4,5.13,0.556820,17.016667,199,0.0,21.529167,23.1732,4.350000,2012,16.757401,...,0.029926,0.112872,0.335116,0.353960,6615.424298,30.657792,17.259663,30.653667,27.830367,30.653667


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1028 entries, 0 to 1027
Data columns (total 28 columns):
PET             1028 non-null float64
VPD             1028 non-null float64
air_temp        1028 non-null float64
doy             1028 non-null int64
precip          1028 non-null float64
soil_temp       1028 non-null float64
sw_in           1028 non-null float64
wind_speed      1028 non-null float64
year            1028 non-null int64
wp_RNET         1028 non-null float64
wp_ch4_gf       1028 non-null float64
wp_co2_gf       1028 non-null float64
wp_er           1028 non-null float64
wp_gpp          1028 non-null float64
wp_h            1028 non-null float64
wp_le           1028 non-null float64
wp_bnd1         1028 non-null float64
wp_bnd2         1028 non-null float64
wp_bnd3         1028 non-null float64
wp_bnd7         1028 non-null float64
wp_evi          1028 non-null float64
wp_lswi2        1028 non-null float64
wp_ndvi         1028 non-null float64
wp_LST.day      1028 no

In [4]:
train_cols = ["precip", "sw_in", "LW_IN.wp", "VPD", "air_temp", "PET", "wind_speed", "wp_LST.day",
             "wp_LST.night", "wp_evi", "wp_ndvi", "wp_lswi2"]
X, Y = exp.featurize(df, train_cols, ["wp_er"])
X, Y, scaler = r.preprocess(X, Y)
X.shape

(1028, 12)

In [5]:
r.random_forests_cross_val(X, Y, feature_names=train_cols)

Running Random Forests Cross Validation...
10-fold CV Acc Mean:  0.941046551294
CV Scores:  0.964686604733, 0.909607111146, 0.957102318473, 0.949188266512, 0.931870976145, 0.954871736931, 0.905600490371, 0.952526849986, 0.961165570957, 0.923845587682
OOB score: 0.941181371975
Feature Importances:
('wp_LST.night', 0.23448265591728668)
('wp_LST.day', 0.19010492331214174)
('PET', 0.12517504951139061)
('air_temp', 0.1215939105092894)
('wind_speed', 0.076058637035192819)
('sw_in', 0.071972864908285986)
('wp_evi', 0.044040170486527808)
('wp_ndvi', 0.038760868247029331)
('wp_lswi2', 0.035489782100324455)
('VPD', 0.03362656864858328)
('LW_IN.wp', 0.026768153619427706)
('precip', 0.0019264157045200158)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=200, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [6]:
r.xgb_trees_cross_val(X, Y, feature_names=train_cols)

Running Gradient Boosted Trees Cross Validation...
10-fold CV Acc Mean:  0.94087125043
CV Scores:  0.960711194933, 0.927084214048, 0.949866105453, 0.942854808169, 0.925390028154, 0.944467549486, 0.917609107702, 0.956666052228, 0.955549800742, 0.928513643389
Feature Importances:
('wp_evi', 0.16555704897698539)
('wp_lswi2', 0.14196599424181403)
('wp_LST.night', 0.10865385519639528)
('wind_speed', 0.10602504984884607)
('wp_LST.day', 0.085558655815499651)
('wp_ndvi', 0.080817712911686712)
('sw_in', 0.072981217080784241)
('PET', 0.06082949195394316)
('VPD', 0.059436102230186777)
('air_temp', 0.057653024968285484)
('LW_IN.wp', 0.051090269554592178)
('precip', 0.009431577220981013)


GradientBoostingRegressor(alpha=0.9, init=None, learning_rate=0.1, loss='ls',
             max_depth=3, max_features='sqrt', max_leaf_nodes=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [7]:
r.svc_cross_val(X, Y)

Running SVC Cross Validation...
10-fold CV Acc Mean:  0.841794765736
CV Scores:  0.862812730564, 0.780355244819, 0.881726447788, 0.84065064491, 0.814474603145, 0.884451926363, 0.831695640657, 0.846426820708, 0.83253147502, 0.84282212339


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [8]:
r.dnn_cross_val(X, Y)

Running Neural Network Cross Validation...
Step #1, avg. loss: 811.19330
Step #501, epoch #50, avg. loss: 21.82231
Step #1001, epoch #100, avg. loss: 6.41256
Step #1501, epoch #150, avg. loss: 4.17782
Step #2001, epoch #200, avg. loss: 3.23327
Step #2501, epoch #250, avg. loss: 2.57253
Step #3001, epoch #300, avg. loss: 2.22069
Step #3501, epoch #350, avg. loss: 1.99497
Step #4001, epoch #400, avg. loss: 1.83910
Step #4501, epoch #450, avg. loss: 1.68049
Step #1, avg. loss: 826.54150
Step #501, epoch #50, avg. loss: 22.34360
Step #1001, epoch #100, avg. loss: 6.63940
Step #1501, epoch #150, avg. loss: 4.33638
Step #2001, epoch #200, avg. loss: 3.11770
Step #2501, epoch #250, avg. loss: 2.50915
Step #3001, epoch #300, avg. loss: 2.26311
Step #3501, epoch #350, avg. loss: 2.03776
Step #4001, epoch #400, avg. loss: 1.91111
Step #4501, epoch #450, avg. loss: 1.82144
Step #1, avg. loss: 750.38928
Step #501, epoch #50, avg. loss: 22.50849
Step #1001, epoch #100, avg. loss: 6.60111
Step #1501

TensorFlowEstimator(batch_size=100, class_weight=None,
          continue_training=False, early_stopping_rounds=None,
          keep_checkpoint_every_n_hours=10000, learning_rate=0.1,
          max_to_keep=5, model_fn=<function tanh_dnn at 0x11352c410>,
          n_classes=0, num_cores=4, optimizer='SGD', steps=5000,
          tf_master='', tf_random_seed=42, verbose=1)